In [238]:
import re
import openpyxl
import pandas as pd
from IPython.display import display
from prettytable import PrettyTable
from functools import reduce
from typing import Literal

col_names = '*'
limit = {}
symbol = '='
table_names = 'table'
tag = ''
using_db = openpyxl.load_workbook('data/test_db.xlsx')
using_dbname = 'test_db'


In [147]:
using_db.sheetnames

['test', 'testTable']

In [148]:
def cartesian_product(*dfs: pd.DataFrame) -> pd.DataFrame:
    return reduce(lambda left,right: pd.merge(left,right,how='outer',on='_'), map(lambda df: df.assign(_=1), dfs)).drop('_', axis=1)

In [149]:
db = {}
for sheet in using_db.worksheets:
    db[sheet.title] = list(sheet.iter_rows(values_only=True))
    db[sheet.title] = pd.DataFrame(db[sheet.title][1:], columns=db[sheet.title][0])
# db_merge = cartesian_product(db['test'].add_prefix('test.'), db['testTable'].add_prefix('testTable.'))
db_merge = cartesian_product(*(db[table].add_prefix(f'{table}.') for table in db.keys()))

In [159]:
db_merge.query('`test.v2` == `testTable.v2` and `test.v3` in (2, 3)')
# db_merge.query('`testTable.v2` == 4')
# db_merge.query('`test.v2` == "4"')

,test.v3,test.v2,testTable.v1,testTable.v2,testTable.v3
27,3,6,侧式,6,3
34,2,4,侧视,4,5


In [160]:
db_merge.query('`testTable.v1`.str.startswith("侧")')

,test.v3,test.v2,testTable.v1,testTable.v2,testTable.v3
2,10,11,侧式,6,3
3,10,11,侧室,3,4
4,10,11,侧视,4,5
7,1,4,侧式,6,3
8,1,4,侧室,3,4
9,1,4,侧视,4,5
12,10,11,侧式,6,3
13,10,11,侧室,3,4
14,10,11,侧视,4,5
17,1,2,侧式,6,3


In [59]:
def iter_rows(ws):  # 表格按行数组形式输出，eg:list(iter_rows(a))
    for row in ws.iter_rows():
        yield [cell.value for cell in row]

In [376]:
from openpyxl import Workbook
def select(col_names: str, 
           table_names: str, 
           constrains: str | None,
           using_db: Workbook) -> pd.DataFrame:
    '''
    Selects the `columns` from the table specified by `table_names` in the database specified by `using_dbname`.
    
    Parameters
    ----------    
    col_names : str
        The columns to select from the table, separated by commas, or '*' for all columns.
    table_names : str
        The name of the table to select from. Can be a single table name or a list of table names separated by commas.
    constrains : str
        The constrains to apply to the selection.
        eg: 'v2=2' means the value of the column v2 must be 2.
    using_db : openpyxl.Workbook
        The database to select from.
    '''
    db = {}
    for sheet in using_db.worksheets:
        db[sheet.title] = list(sheet.iter_rows(values_only=True))
        db[sheet.title] = pd.DataFrame(db[sheet.title][1:], columns=db[sheet.title][0])

    ##############################
    # FROM

    table_name_list = [table.strip() for table in table_names.split(',') if table.strip() in db.keys()]
    if (len(table_name_list) == 0):
        return print("表格不存在")
    elif len(table_name_list) == 1:
        df = db[table_name_list[0]]
    else:
        df = cartesian_product(*(db[table].add_prefix(f'{table}.') for table in db.keys()))

    ##############################
    # SELECT

    if col_names != '*':
        col_name_list = []
        for col in col_names.split(','):
            col = col.strip()
            exist = False
            if len(table_name_list) == 1:
                col_name = col
                if col_name in df.columns:
                    col_name_list.append(col_name)
                    exist = True
            else:
                for table in table_name_list:
                    col_name = f'{table}.{col}'
                    if col_name in df.columns:
                        col_name_list.append(col_name)
                        exist = True
            if not exist:
                return print("列名不存在")
            
        df = df[col_name_list]

    ##############################
    # WHERE

    if constrains:
        constrains = re.sub(r"(\w+\.\w+)", lambda match: f"`{match.group(1)}`", constrains)
        constrains = re.sub(r"(\S+) like (\S+)", lambda match: f'{match.group(1)}.str.match({match.group(2).replace("%", ".*").replace("_", ".")})', constrains)
        try:
            df = df.query(constrains)
        except Exception as e:
            return print(f'查询条件错误：{e}')


    ##############################
    # OUTPUT

    tb = PrettyTable()
    tb.field_names = df.columns
    for index, row in df.iterrows():
        tb.add_row(row)
    tb.reversesort = True
    # print(tb)
    return tb


select(
    col_names='*', 
    table_names='test, testTable',
    constrains='test.v2 == testTable.v2 and test.v3 in (2, 3) and testTable.v1 like "侧%"',
    using_db=openpyxl.load_workbook('data/test_db.xlsx')
)

# select(
#     col_names='v2', 
#     table_names='test',
#     constrains='v2 in (2, 3)',
#     using_db=openpyxl.load_workbook('data/test_db.xlsx')
# )


test.v3,test.v2,testTable.v1,testTable.v2,testTable.v3
3,6,侧式,6,3
2,4,侧视,4,5


In [365]:
df = pd.DataFrame({'A': ['asasddfc', 'abc', 'zxcv'], 'B': [1, 2, 3]})
df.query('A.str.match("..a.*c")')

,A,B
0,asasddfc,1


In [369]:
import re

# 原始的 WHERE 子句
constrains = "test.v2 == testTable.v2 and test.v3 like some% and test.v2 == 4"

constrains = re.sub(r"(\w+\.\w+)", lambda match: f"`{match.group(1)}`", constrains)
# 正则表达式，用于匹配类似于 'LIKE table.column' 的模式
pattern = r"(\S+) like (\S+)"

# 替换函数，为匹配的列名添加反引号
def replace_with_backticks(match):
    return f'{match.group(1)}.str.match("{match.group(2).replace("%", ".*").replace("_", ".")}")'

# 执行替换
modified_where_clause = re.sub(pattern, replace_with_backticks, constrains)

print(modified_where_clause)


('`test.v3`', 'some%')
`test.v2` == `testTable.v2` and `test.v3`.str.match("some.*") and `test.v2` == 4


In [306]:
pattern = r"select\s+(.*?)\s+from\s+([\w\s,]+?)(?:\s+where\s+(.+))?$"
pattern = re.compile(pattern, re.IGNORECASE)
sql_query = 'select * from testTable, test where v3>1'
res = re.match(pattern, sql_query)

if res:
    col_names, tables, where_clause = res.groups()
    print("Columns:", col_names)
    print("Tables:", tables)
    print("Where clause:", where_clause)
else:
    print("No match found")


Columns: *
Tables: testTable, test
Where clause: v3>1
